In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import requests
from datetime import datetime
import psycopg


ModuleNotFoundError: No module named 'seaborn'


# Load data from OpenWeatherMap API

In [8]:
api_k = '66e13da3899712beb58967ca82ebfefa' 
# city = 'Vancouver,CA'
# Latitude
lat = '49.28'
# Longitude - use negative for places west of greenwich
lon = '-123.12'
# of days returned in API response
# cnt = '7'

# Can't request data since I'm on Free plan - doesn't allow me to draw data from paid subscriptions (16days/daily)
# url = f"https://api.openweathermap.org/data/2.5/forecast/daily?lat={lat}&lon={lon}&cnt={cnt}&appid={api_k}"


# Getting One Call
# One Call API 3.0 contains 4 endpoints and provides access to various data:

# Current weather and forecasts:
# minute forecast for 1 hour
# hourly forecast for 48 hours
# daily forecast for 8 days
# and government weather alerts
# Weather data for any timestamp for 46+ years historical archive and 4 days ahead forecast
# Daily aggregation of weather data for 46+ years archive and 1.5 years ahead forecast
# Weather overview with a human-readable weather summary for today and tomorrow's forecast, utilizing OpenWeather AI technologies

url = f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&appid={api_k}"

response = requests.get(url)


In [9]:
data = response.json()
json_str = json.dumps(data, indent = 4)
print(json_str)

{
    "lat": 49.28,
    "lon": -123.12,
    "timezone": "America/Vancouver",
    "timezone_offset": -28800,
    "current": {
        "dt": 1735931186,
        "sunrise": 1735920447,
        "sunset": 1735950391,
        "temp": 277.5,
        "feels_like": 276.3,
        "pressure": 1007,
        "humidity": 92,
        "dew_point": 276.32,
        "uvi": 0.19,
        "clouds": 100,
        "visibility": 10000,
        "wind_speed": 1.54,
        "wind_deg": 70,
        "weather": [
            {
                "id": 804,
                "main": "Clouds",
                "description": "overcast clouds",
                "icon": "04d"
            }
        ]
    },
    "minutely": [
        {
            "dt": 1735931220,
            "precipitation": 0
        },
        {
            "dt": 1735931280,
            "precipitation": 0
        },
        {
            "dt": 1735931340,
            "precipitation": 0
        },
        {
            "dt": 1735931400,
            "precipit

## Connect to local db

In [4]:
# conn = psycopg.connect("dbname=myinner_db host=localhost user=jasoncao password=P0stgr5q168!", options="-c client_encoding=UTF8")
# conn = psycopg.connect("dbname=w_db_inner host=localhost user=jasoncao password=P0stgr5q168!", options="-c client_encoding=UTF8")
conn = psycopg.connect("dbname=weather_db" host=localhost user=postgres password=postgrestest, options="-c client_encoding=UTF8")

cur = conn.cursor()


# Transforming Data

Create dataframe for extracted data.

In [5]:
def kelvin_to_fahrenheit(temp_in_kelvin):
    temp_in_fahrenheit = (temp_in_kelvin - 273.15) * (9/5) + 32
    return round(temp_in_fahrenheit, 3)

In [32]:
# Transform code from json into a table
if response.status_code == 200:
    data = response.json()
#     weather_data = []
#     for hour in data['list']:
#         date_time = datetime.utcfromtimestamp(hour['dt']).strftime('%Y-%m-%d %H:%M:%S')
#         temp = kelvin_to_fahrenheit(hour['main']['temp'])
#         feels_like = kelvin_to_fahrenheit(hour['main']['feels_like'])
#         pressure = hour['main']['pressure']
#         humidity = hour['main']['humidity']
#         weather_main = hour['weather'][0]['main']
#         weather_description = hour['weather'][0]['description']
#         wind_speed = hour['wind']['speed']
#         wind_direction = hour['wind']['deg']
#         cloudiness = hour['clouds']['all']
#         rain_volume = hour.get('rain', {}).get('3h',0)
#         snow_volume = hour.get('snow', {}).get('3h',0)
        
#         weather_data.append({
#             'DateTime': date_time,
#             'Temperature': temp,
#             'Feels Like_temp': feels_like,
#             'Pressure(hPa)': pressure,
#             'Humidity_percent': humidity,
#             'Weather': weather_main,
#             'Weather Description': weather_description,
#             'Wind Speed': wind_speed,
#             'Wind Direction': wind_direction,
#             'Cloudiness': cloudiness,
#             'Rain Volume(mm)': rain_volume,
#             'Snow Volume(mm)': snow_volume
#         })
    weather_data = []
    for day in data['daily']:
        date_time = datetime.utcfromtimestamp(day['dt']).strftime('%Y-%m-%d')
        temp_day = kelvin_to_fahrenheit(day['temp']['day'])
        temp_min = kelvin_to_fahrenheit(day['temp']['min'])
        temp_max = kelvin_to_fahrenheit(day['temp']['max'])
        feels_like_day = kelvin_to_fahrenheit(day['feels_like']['day'])
        pressure = day['pressure']
        humidity = day['humidity']
        weather_main = day['weather'][0]['main']
        weather_description = day['weather'][0]['description']
        wind_speed = day['wind_speed']
        wind_direction = day['wind_deg']
        cloudiness = day['clouds']
        rain_volume = day.get('rain', 0)
        snow_volume = day.get('snow', 0)

        weather_data.append({
            'Date': date_time,
            'Temperature_F': temp_day,
            'Temperature_Min_F': temp_min,
            'Temperature_Max_F': temp_max,
            'Temp_Feels_Like_F': feels_like_day,
            'Pressure_hPa': pressure,
            'Humidity_percent': humidity,
            'Weather': weather_main,
            'Weather_Description': weather_description,
            'Wind_Speed_m_per_s': wind_speed,
            'Wind_Direction_degrees': wind_direction,
            'Cloudiness_percent': cloudiness,
            'Rain_Volume_mm': rain_volume,
            'Snow_Volume_mm': snow_volume
        })

    # Convert to a pandas DataFrame
    df = pd.DataFrame(weather_data)

    # Print the DataFrame or save it to a file
    print(df)

         Date  Temperature_F  Temperature_Min_F  Temperature_Max_F  \
0  2025-01-03         40.010             39.668             42.998   
1  2025-01-04         44.006             41.666             44.258   
2  2025-01-05         44.258             42.926             44.384   
3  2025-01-06         44.636             39.398             44.636   
4  2025-01-07         42.728             37.814             42.728   
5  2025-01-08         43.412             38.192             43.412   
6  2025-01-09         44.654             40.136             44.654   
7  2025-01-10         41.288             41.288             42.440   

   Temp_Feels_Like_F  Pressure_hPa  Humidity_percent Weather  \
0             35.978          1007                91    Rain   
1             42.638          1017                87    Rain   
2             42.836          1023                91    Rain   
3             43.610          1031                68  Clouds   
4             42.728          1033               

### Save initial results to CSV file

In [49]:
df.to_csv('OWM_One_Call_API_3.0_Vancouver_data.csv', index = False)

### PostgreSQL Data Analysis

#### Create Table & Query All Results

In [26]:
path = response.json()
cur.execute("""DROP TABLE IF EXISTS Weather_Data""")
cur.execute("""
  CREATE TABLE IF NOT EXISTS Weather_Data (
    id SERIAL PRIMARY KEY,
    Date DATE,
    Temperature_F TEXT,
    Temperature_Min_F TEXT,
    Temperature_Max_F TEXT,
    Temp_Feels_Like_F TEXT,
    Pressure_hPa TEXT,
    Humidity_percent TEXT,
    Weather TEXT,
    Weather_Description TEXT,
    Wind_Speed_m_per_s TEXT,
    Wind_Direction_degrees TEXT,
    Cloudiness_percent TEXT,
    Rain_Volume_mm TEXT,
    Snow_Volume_mm TEXT
  )
""")
conn.commit()

In [57]:
weather_list = list(df[['Date', 'Temperature_F', 'Temperature_Min_F', 'Temperature_Max_F',
                       'Temp_Feels_Like_F','Pressure_hPa','Humidity_percent',
                       'Weather','Weather_Description','Wind_Speed_m_per_s',
                       'Wind_Direction_degrees','Cloudiness_percent','Rain_Volume_mm',
                       'Snow_Volume_mm']].itertuples(index=False, name=None))
insert_query = """
        INSERT INTO Weather_Data (Date, Temperature_F, Temperature_Min_F, Temperature_Max_F,
        Temp_Feels_Like_F, Pressure_hPa, Humidity_percent, Weather,Weather_Description,
        Wind_Speed_m_per_s, Wind_Direction_degrees, Cloudiness_percent, Rain_Volume_mm, Snow_Volume_mm)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) 
    """ # The ammt of %s must match the amt of attributes added into the table
cur.executemany(insert_query, weather_list)

In [58]:
cur.execute("SELECT * FROM Weather_Data")
rows = cur.fetchall()
query_df = pd.DataFrame(rows, columns=['id','Date', 'Temperature_F', 'Temperature_Min_F', 'Temperature_Max_F',
                       'Temp_Feels_Like_F','Pressure_hPa','Humidity_percent',
                       'Weather','Weather_Description','Wind_Speed_m_per_s',
                       'Wind_Direction_degrees','Cloudiness_percent','Rain_Volume_mm',
                       'Snow_Volume_mm'])
print(query_df)

   id        Date Temperature_F Temperature_Min_F Temperature_Max_F  \
0  25  2025-01-03         40.01            39.668            42.998   
1  26  2025-01-04        44.006            41.666            44.258   
2  27  2025-01-05        44.258            42.926            44.384   
3  28  2025-01-06        44.636            39.398            44.636   
4  29  2025-01-07        42.728            37.814            42.728   
5  30  2025-01-08        43.412            38.192            43.412   
6  31  2025-01-09        44.654            40.136            44.654   
7  32  2025-01-10        41.288            41.288             42.44   

  Temp_Feels_Like_F Pressure_hPa Humidity_percent Weather Weather_Description  \
0            35.978         1007               91    Rain       moderate rain   
1            42.638         1017               87    Rain       moderate rain   
2            42.836         1023               91    Rain       moderate rain   
3             43.61         1031    

##### Speed & Memory Analysis

In [59]:
cur.execute("EXPLAIN ANALYZE SELECT * FROM Weather_Data")
query_analysis = cur.fetchall()
# query_analysis
print(query_analysis)

[('Seq Scan on weather_data  (cost=0.00..11.80 rows=180 width=424) (actual time=0.023..0.050 rows=8 loops=1)',), ('Planning Time: 0.075 ms',), ('Execution Time: 0.110 ms',)]


#### Query Largest Range of Difference in Temperatures

In [60]:
cur.execute("SELECT Date, CAST(Temperature_Max_F AS FLOAT) - CAST(Temperature_Min_F AS FLOAT) AS Temp_Diff FROM Weather_Data ORDER BY Temp_Diff DESC")
rows = cur.fetchall()
diff_query_df = pd.DataFrame(rows, columns=['Date', 'Temp_Diff'])
print(diff_query_df)

         Date  Temp_Diff
0  2025-01-06      5.238
1  2025-01-08      5.220
2  2025-01-07      4.914
3  2025-01-09      4.518
4  2025-01-03      3.330
5  2025-01-04      2.592
6  2025-01-05      1.458
7  2025-01-10      1.152


In [ ]:
# Use when postgresql commands error:
conn.rollback()

## Close db connection

In [ ]:
cur.close()
conn.close()
